In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as se
%matplotlib inline

In [0]:
from google.colab import files
uploaded = files.upload()

Saving test_values.csv to test_values.csv
Saving train_labels.csv to train_labels.csv
Saving train_values.csv to train_values.csv


In [0]:

train=pd.read_csv('train_values.csv')
labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_values.csv')
labels.head()

,row_id,accepted
0,0,1
1,1,0
2,2,1
3,3,1
4,4,1


In [0]:
train.rename(columns={'number_of_owner-occupied_units':'number_of_owner_occupied_units'},inplace=True)

In [0]:
train.applicant_income=train.applicant_income.fillna(np.mean(train.applicant_income))
train.number_of_1_to_4_family_units =train.number_of_1_to_4_family_units.fillna(np.mean(train.number_of_1_to_4_family_units))
train.number_of_owner_occupied_units =train.number_of_owner_occupied_units.fillna(np.mean(train.number_of_owner_occupied_units))
train.ffiecmedian_family_income =train.ffiecmedian_family_income.fillna(np.mean(train.ffiecmedian_family_income))
train.minority_population_pct =train.minority_population_pct.fillna(np.mean(train.minority_population_pct))
train.population =train.population.fillna(np.mean(train.population))
train.tract_to_msa_md_income_pct =train.tract_to_msa_md_income_pct.fillna(np.mean(train.tract_to_msa_md_income_pct))

In [0]:
test.rename(columns={'number_of_owner-occupied_units':'number_of_owner_occupied_units'},inplace=True)

In [0]:
test.applicant_income=train.applicant_income.fillna(np.mean(train.applicant_income))
test.number_of_1_to_4_family_units =train.number_of_1_to_4_family_units.fillna(np.mean(train.number_of_1_to_4_family_units))
test.number_of_owner_occupied_units =train.number_of_owner_occupied_units.fillna(np.mean(train.number_of_owner_occupied_units))
test.ffiecmedian_family_income =train.ffiecmedian_family_income.fillna(np.mean(train.ffiecmedian_family_income))
test.minority_population_pct =train.minority_population_pct.fillna(np.mean(train.minority_population_pct))
test.population =train.population.fillna(np.mean(train.population))
test.tract_to_msa_md_income_pct =train.tract_to_msa_md_income_pct.fillna(np.mean(train.tract_to_msa_md_income_pct))

In [0]:
#All=pd.concat([train,labels.accepted],axis=1)
train=pd.merge(train,labels, how='inner', on='row_id')
train.head(3)
#All.head()

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,applicant_ethnicity,applicant_race,applicant_sex,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner_occupied_units,number_of_1_to_4_family_units,lender,co_applicant,accepted
0,0,3,1,1,1,70.0,3,18,37,246,2,5,1,24.0,6203.0,44.230,60588.0,50.933,716.0,2642.0,4536,False,1
1,1,1,1,3,1,178.0,3,369,52,299,1,5,1,57.0,5774.0,15.905,54821.0,100.000,1622.0,2108.0,2458,False,0
2,2,2,1,3,1,163.0,3,16,10,306,2,5,1,67.0,6094.0,61.270,67719.0,100.000,760.0,1048.0,5710,False,1


In [0]:
catX = train.drop(['loan_amount','applicant_income','population','minority_population_pct',
                   'ffiecmedian_family_income','tract_to_msa_md_income_pct','number_of_owner_occupied_units',
                   'number_of_1_to_4_family_units'], axis=1) 
catX.head()

,row_id,loan_type,property_type,loan_purpose,occupancy,preapproval,msa_md,state_code,county_code,applicant_ethnicity,applicant_race,applicant_sex,lender,co_applicant,accepted
0,0,3,1,1,1,3,18,37,246,2,5,1,4536,False,1
1,1,1,1,3,1,3,369,52,299,1,5,1,2458,False,0
2,2,2,1,3,1,3,16,10,306,2,5,1,5710,False,1
3,3,1,1,1,1,1,305,47,180,2,5,1,5888,True,1
4,4,1,1,1,1,3,24,37,20,2,3,2,289,False,1


In [0]:
bool_data = train.select_dtypes(include=['bool']).copy()
new_data_lc = bool_data.copy()
catX['co_applicant'] = new_data_lc['co_applicant'].astype('category')
catX['co_applicant'] = catX['co_applicant'].cat.codes
catX.head()


,row_id,loan_type,property_type,loan_purpose,occupancy,preapproval,msa_md,state_code,county_code,applicant_ethnicity,applicant_race,applicant_sex,lender,co_applicant,accepted
0,0,3,1,1,1,3,18,37,246,2,5,1,4536,0,1
1,1,1,1,3,1,3,369,52,299,1,5,1,2458,0,0
2,2,2,1,3,1,3,16,10,306,2,5,1,5710,0,1
3,3,1,1,1,1,1,305,47,180,2,5,1,5888,1,1
4,4,1,1,1,1,3,24,37,20,2,3,2,289,0,1


In [0]:
numy = train.loc[:,['loan_amount','applicant_income','population','minority_population_pct',
                   'ffiecmedian_family_income','tract_to_msa_md_income_pct','number_of_owner_occupied_units',
                   'number_of_1_to_4_family_units']]
numy.head()

,loan_amount,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner_occupied_units,number_of_1_to_4_family_units
0,70.0,24.0,6203.0,44.230,60588.0,50.933,716.0,2642.0
1,178.0,57.0,5774.0,15.905,54821.0,100.000,1622.0,2108.0
2,163.0,67.0,6094.0,61.270,67719.0,100.000,760.0,1048.0
3,155.0,105.0,6667.0,6.246,78439.0,100.000,2025.0,2299.0
4,305.0,71.0,6732.0,100.000,63075.0,82.200,1464.0,1847.0


In [0]:
numy.population=np.log(numy.population)
numy.loan_amount=np.log(numy.loan_amount)
numy.applicant_income=np.log(numy.applicant_income)
numy.minority_population_pct=np.log(numy.minority_population_pct)
numy.ffiecmedian_family_income=np.log(numy.ffiecmedian_family_income)
numy.tract_to_msa_md_income_pct=np.log(numy.tract_to_msa_md_income_pct)
numy.number_of_owner_occupied_units=np.log(numy.number_of_owner_occupied_units)
numy.number_of_1_to_4_family_units=np.log(numy.number_of_1_to_4_family_units)


In [0]:
All=pd.concat([numy, catX], axis=1)
All.head(3)

,loan_amount,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner_occupied_units,number_of_1_to_4_family_units,row_id,loan_type,property_type,loan_purpose,occupancy,preapproval,msa_md,state_code,county_code,applicant_ethnicity,applicant_race,applicant_sex,lender,co_applicant,accepted
0,4.248495,3.178054,8.732788,3.789403,11.011852,3.930511,6.573680,7.879291,0,3,1,1,1,3,18,37,246,2,5,1,4536,0,1
1,5.181784,4.043051,8.661120,2.766634,10.911829,4.605170,7.391415,7.653495,1,1,1,3,1,3,369,52,299,1,5,1,2458,0,0
2,5.093750,4.204693,8.715060,4.115290,11.123122,4.605170,6.633318,6.954639,2,2,1,3,1,3,16,10,306,2,5,1,5710,0,1


In [0]:
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split

In [0]:
Unused=['tract_to_msa_md_income_pct','number_of_owner_occupied_units','msa_md','county_code','applicant_race','minority_population_pct','accepted']
#feats=['loan_type','number_of_1_to_4_family_units',
#       'property_type', 'loan_purpose', 'occupancy','lender','co_applicant',
#       'loan_amount', 'preapproval', 'state_code','applicant_ethnicity','applicant_sex','applicant_income','population','ffiecmedian_family_income']
#list = ['url','shares']
X = All.drop(Unused, axis = 1)
#x = x[feats]
y = labels.loc[:,'accepted']

In [0]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=42)

In [0]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from sklearn.metrics import mean_squared_error as mse

In [0]:
!pip install catboost

     |████████████████████████████████| 62.6MB 36.5MB/s 


In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC  
from catboost import CatBoostClassifier

In [0]:
svc=SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)

In [0]:
#neigh.fit(X, y)
svc.fit(X_train, y_train)

# Prediction on the training dataset
y_tr_pred = svc.predict(X_train)

# Classification repport on the training dataset:    
print('Classification repport:\n',classification_report(y_train, y_tr_pred))
# Accuracy score on the training dataset
print('Accuracy score:\n',svc.score(X_train, y_train))

In [0]:
# Test the learner, Log, with the test dataset

## Prediction on the training dataset
y_te_pred = svc.predict(X_test)

# Classification repport on the training dataset:
print('Classification repport:\n',classification_report(y_test, y_te_pred))

# Accuracy score on the training dataset
print('Accuracy score:\n',svc.score(X_test, y_test))

In [0]:
from sklearn.model_selection import cross_val_predict, cross_val_score

# List of metrics that helps in assessing the performance of the learner for different metric
cv_metric = ['accuracy', 'precision_macro', 'recall_macro']
for i in range(len(cv_metric)):
    scores = cross_val_score(svc, X, y, scoring=cv_metric[i],cv=5)
    print('\nMetric used: ', cv_metric[i])
    print('Cross validation scores:', scores)
    print('Mean of the scores: ', scores.mean(),'\n with standard deviation', scores.std())